In [2]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import torch
import torchvision
import torchvision.transforms as tvt
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10, CIFAR100, SVHN, Places365
from torch.utils.data import ConcatDataset, DataLoader

from pytorch_ood.dataset.img import (
    LSUNCrop,
    Textures,
    TinyImageNetCrop,
    GaussianNoise
)
from pytorch_ood.detector import (
    ODIN,
    EnergyBased,
    Entropy,
    KLMatching,
    Mahalanobis,
    MaxLogit,
    MaxSoftmax,
    ViM,
)
from pytorch_ood.model import WideResNet
from pytorch_ood.utils import OODMetrics, ToRGB, ToUnknown

torch.manual_seed(1234)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([
    tvt.Resize(size=(32, 32)),
    ToRGB(),
    tvt.ToTensor(),
    tvt.Normalize(std=std, mean=mean)
])

In [4]:
def calculate(test_loader, detector):
    metrics = OODMetrics() # Evaluate Detectors
    
    print("Total count:", len(test_loader))
    cnt = 0
    for x, y in test_loader:
        metrics.update(detector(x.cuda()), y)
        cnt += 1
        if (cnt + 1) % 50 == 0:
            print(f"[{cnt}/{len(test_loader)}]")
    
    r = metrics.compute()
    print(r)
    return r

In [5]:
# Download checks.
# dataset_out_test = Places365(root="data", split="val", small=True, transform=trans, target_transform=ToUnknown(), download=True)

In [6]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available! Number of GPUs: {torch.cuda.device_count()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")


CUDA is available! Number of GPUs: 1
GPU Name: NVIDIA GeForce RTX 3090


In [7]:
def test_with_all_datasets(detector, base_dataset, transform=trans, batch_size=128, num_workers=20):
    dataset_names = ["SVHN", "Textures", "LSUNCrop", "TinyImageNetCrop", "Places365", "GaussianNoise"]
    results = []

    for name in dataset_names:
        if name == "SVHN":
            dataset_out_test = SVHN(root="data", split="test", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "Textures":
            dataset_out_test = Textures(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "LSUNCrop":
            dataset_out_test = LSUNCrop(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "TinyImageNetCrop":
            dataset_out_test = TinyImageNetCrop(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "Places365":
            dataset_out_test = Places365(root="data", split="val", small=True, transform=transform, target_transform=ToUnknown())
        elif name == "GaussianNoise":
            dataset_out_test = GaussianNoise(length=10000, transform=transform, target_transform=ToUnknown())
        else:
            continue
        
        test_loader = DataLoader(torch.utils.data.ConcatDataset([base_dataset, dataset_out_test]), batch_size=batch_size, num_workers=num_workers)
        print()
        print(name)
        results.append(calculate(test_loader, detector))
    
    return results

In [8]:
from torch import nn

class OODDector(nn.Module):
    def __init__(self, num_classes=10, multi_class=True):
        super().__init__()
        self.fc_out = nn.Linear(8192, 2048)
        self.relu = nn.ReLU()
        if multi_class:
            k = num_classes + 1
        else:
            k = 1
        self.out_layer = nn.Linear(2048, k)

    def forward(self, x):
        x = x.flatten(1)
        x = self.fc_out(x)
        x = self.relu(x)
        x = self.out_layer(x)
        return x

In [9]:
import torchvision.datasets as dset
import numpy as np
import cv2
from typing import Any, Tuple
from PIL import Image
import random

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

class CIFAR10withOOD(dset.CIFAR10):
    
    torch.manual_seed(42)
    def __init__(self, root, batch_size, ood_files=None, download=True, train=True, transform=None,
                 ood_transform=None, use_patch_aug=True, multiclass=True):
        self.root = root
        self.ood_files = ood_files
        self.num_classes = 10
        self.batch_size = batch_size
        self.ood_transform = ood_transform
        self.use_patch_aug = use_patch_aug
        self.multiclass = multiclass
        if multiclass:
            self.ood_prob = (self.batch_size / (self.num_classes + 1) * 0.01)
        else:
            self.ood_prob = 0.5
        super().__init__(root, download=download, train=train, transform=transform)

    def extract_blocks(self, image, mask):
        maximum_score = 8 * 8
        patch_size = 8
        stride = patch_size // 2
        patch_images = []
        mask_score = []
        for i in range(0, 28, stride):
            for j in range(0, 28, stride):
                if (j + patch_size) > 32:
                    break
                patch_image = image[i:i + patch_size, j:j + patch_size]
                patch_score = maximum_score - np.sum(mask[i:i + patch_size, j:j + patch_size])
                if patch_score == maximum_score:
                    if np.random.uniform() >= 0.5:
                        resize_image = cv2.resize(patch_image, (32, 32))
                    else:
                        resize_image = np.tile(patch_image, [4, 4, 1])
                    return resize_image
                patch_images.append(patch_image)
                mask_score.append(patch_score)

            if (i + patch_size) > 32:
                break

        max_idx = np.argmax(mask_score, axis=0)
        max_patch = patch_images[max_idx]
        if np.random.uniform() >= 0.5:
            resize_image = cv2.resize(max_patch, (32, 32))
        else:
            resize_image = np.tile(max_patch, [4, 4, 1])
        return resize_image

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if np.random.uniform() <= self.ood_prob:
            rand_idx = np.random.randint(0, len(self.ood_files))
            img_file = self.ood_files[rand_idx]
            img = np.array(Image.open(img_file).convert('RGB')).astype(np.uint8)
            target = self.num_classes

            if self.use_patch_aug and np.random.uniform() >= 0.5:
                mask_file = img_file.replace("_train.png", "_mask.png")
                mask = np.array(Image.open(mask_file)) / 255.
                mask = np.uint8(mask)
                img = self.extract_blocks(img, mask)

            img = Image.fromarray(img)
            if self.ood_transform is not None:
                img = self.ood_transform(img)

        else:
            img, target = self.data[index], self.targets[index]
            img = Image.fromarray(img)

            if self.transform is not None:
                img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

In [10]:
def get_all_blocks(x, model):
    out_list = []
    out = model.conv1(x)
    out = model.block1(out)
    out_list.append(out)
    out = model.block2(out)
    out_list.append(out)
    out = model.block3(out)
    out = model.relu(model.bn1(out))
    out_list.append(out)
    return out_list[2].flatten(1)

In [11]:
from abc import ABC, abstractmethod
from typing import TypeVar

from torch import Tensor
from torch.utils.data import DataLoader

Self = TypeVar("Self")


class RequiresFittingException(Exception):
    """
    Raised when predict is called on a detector that has not been fitted.
    """

    def __init__(self, msg="You have to call fit() before predict()"):
        super(RequiresFittingException, self).__init__(msg)


class ModelNotSetException(ValueError):
    """
    Raised when predict() is called but no model was given.
    """

    def __init__(self, msg="When using predict(), model must not be None"):
        super(ModelNotSetException, self).__init__(msg)


class Detector(ABC):
    """
    Abstract Base Class for an Out-of-Distribution Detector
    """

    def __call__(self, *args, **kwargs) -> Tensor:
        """
        Forwards to predict
        """
        return self.predict(*args, **kwargs)

    @abstractmethod
    def fit(self: Self, data_loader: DataLoader) -> Self:
        """
        Fit the detector to a dataset. Some methods require this.
        :param data_loader: dataset to fit on. This is usually the training dataset.
        :raise ModelNotSetException: if model was not set
        """
        raise NotImplementedError

    @abstractmethod
    def fit_features(self: Self, x: Tensor, y: Tensor) -> Self:
        """
        Fit the detector directly on features. Some methods require this.
        :param x: training features to use for fitting.
        :param y: corresponding class labels.
        """
        raise NotImplementedError

    @abstractmethod
    def predict(self, x: Tensor) -> Tensor:
        """
        Calculates outlier scores. Inputs will be passed through the model.
        :param x: batch of data
        :return: outlier scores for points
        :raise RequiresFitException: if detector has to be fitted to some data
        :raise ModelNotSetException: if model was not set
        """
        raise NotImplementedError

    @abstractmethod
    def predict_features(self, x: Tensor) -> Tensor:
        """
        Calculates outlier scores based on features.
        :param x: batch of data
        :return: outlier scores for points
        :raise RequiresFitException: if detector has to be fitted to some data
        """
        raise NotImplementedError

In [12]:
from typing import Optional, TypeVar

from torch import Tensor
from torch.nn import Module

Self = TypeVar("Self")


class Mine(Detector):
    """
    Implements the Maximum Softmax Probability (MSP) Thresholding baseline for OOD detection.
    Optionally, implements temperature scaling, which divides the logits by a constant temperature :math:`T`
    before calculating the softmax.
    .. math:: - \\max_y \\sigma_y(f(x) / T)
    where :math:`\\sigma` is the softmax function and :math:`\\sigma_y`  indicates the :math:`y^{th}` value of the
    resulting probability vector.
    :see Paper:
        `ArXiv <https://arxiv.org/abs/1610.02136>`_
    :see Implementation:
        `GitHub <https://github.com/hendrycks/error-detection>`_
    """

    def __init__(self, model: Module, ood_detector: Module, t: Optional[float] = 1.0):
        """
        :param model: neural network to use
        :param t: temperature value :math:`T`. Default is 1.
        """
        super(Mine, self).__init__()
        self.t = t
        self.ood_detector = ood_detector
        self.model = model

    def predict(self, x: Tensor) -> Tensor:
        """
        :param x: input, will be passed through model
        """
        if self.model is None:
            raise ModelNotSetException

        features = get_all_blocks(x, self.model)
        pred = self.ood_detector(features.detach())
        return pred 


    def fit(self: Self, *args, **kwargs) -> Self:
        """
        Not required
        """
        return self

    def fit_features(self: Self, *args, **kwargs) -> Self:
        """
        Not required
        """
        return self

    def predict_features(self, logits: Tensor) -> Tensor:
        """
        :param logits: logits given by the model
        """
        return MaxSoftmax.score(logits, self.t)

In [13]:
import torchvision.transforms as trn
import torch.nn.functional as F
import glob
from tqdm import tqdm

def train_ood_detector(ood_dataset_path, model, batch_size=128, epochs=10, seed=42):

    #torch.manual_seed(seed)
    
    ood_transform = [
        trn.RandomHorizontalFlip(),
        trn.RandomAffine(0, (0.0, 0.0), (1.0, 1.5)),
        trn.RandomAutocontrast(0.2),
        trn.RandomInvert(0.2),
        trn.ToTensor(),
        trn.Normalize(mean, std),
    ]
    transform_ood = trn.Compose(ood_transform)
    
    ind_test_data = dset.CIFAR10(f'data', download=True, train=False, transform=trans)
    num_classes = 10
    batch_size = batch_size
    epochs = epochs
    
    ind_test_loader = DataLoader(ind_test_data, batch_size=batch_size // 2, shuffle=False, num_workers=8)
    
    args_multi_class = 0
    
    if args_multi_class:
        multi_class = True
        criterion = torch.nn.CrossEntropyLoss()
    else:
        multi_class = False
        criterion = F.binary_cross_entropy_with_logits
    
    ood_detector = OODDector(num_classes=num_classes, multi_class=multi_class)
    
    optimizer = torch.optim.SGD(ood_detector.parameters(), lr=0.01, weight_decay=0.0004, momentum=0.8, nesterov=True)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    if torch.cuda.is_available():
        ood_detector.cuda()
    
    all_files = []
    if isinstance(ood_dataset_path, list):  
        for path in ood_dataset_path:
            all_files.extend(glob.glob(os.path.join(path, "*.png")))
    else: 
        all_files.extend(glob.glob(os.path.join(ood_dataset_path, "*.png")))
    
    print(f"CAM-based OOD NUM : {len(all_files)}")
    
    # ood train dataset
    concat_dataset = CIFAR10withOOD(f'data', batch_size, ood_files=all_files,
                                    download=True, train=True, transform=trans, ood_transform=transform_ood,
                                    use_patch_aug=True, multiclass=multi_class)
    train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=20, drop_last=True)
    
    # train ood classifier
    for epoch in range(epochs):
        loader = tqdm(train_loader, disable=False, ascii=True)
        loader.set_description('[%s %03d/%03d]' % ('train', epoch + 1, epochs))
        cnt = 0
        for batch_idx, (x_data, y_data) in enumerate(loader):
            cnt += batch_size
    
            if torch.cuda.is_available():
                x_data = x_data.cuda()
                y_data = y_data.cuda()
    
            optimizer.zero_grad()
            features = get_all_blocks(x_data, model)
            # print(features.detach())
            pred = ood_detector(features.detach())
            # print(pred)
            if multi_class:
                loss = criterion(pred, y_data.view(-1).cuda().to(torch.long))
            else:
                y_data = torch.where(y_data == num_classes, torch.ones_like(y_data), torch.zeros_like(y_data))
                loss = criterion(pred, y_data.view(-1, 1).cuda().to(torch.float32))
            
            loss.backward()
            optimizer.step()
    
        scheduler.step()

    return ood_detector

### MLM

In [70]:
MLM_path = "/home/dasol/DMR/DMR/CIFAR10_OOD_training_images_using_MLM_without_DMR"
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
MLM = train_ood_detector(MLM_path, model, batch_size=128, epochs=10)

Files already downloaded and verified
CAM-based OOD NUM : 50000
Files already downloaded and verified


[train 010/010]: 100%|##############################################| 390/390 [00:04<00:00, 78.48it/s]


In [71]:
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
detector = Mine(model, MLM)
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_AUPR = np.mean([r['AUPR-IN'] for r in results])

print("AUROC:", mean_AUROC)
print("AURR:", mean_AUPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.9671512842178345, 'AUPR-IN': 0.9872857928276062, 'AUPR-OUT': 0.9065738320350647, 'FPR95TPR': 0.17579999566078186}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.9040053486824036, 'AUPR-IN': 0.8908250331878662, 'AUPR-OUT': 0.9053792953491211, 'FPR95TPR': 0.6421999931335449}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9802474975585938, 'AUPR-IN': 0.9724568724632263, 'AUPR-OUT': 0.9830540418624878, 'FPR95TPR': 0.08089999854564667}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9673318862915039, 'AUPR-IN': 0.9621306657791138, 'AUPR-OUT': 0.9692314267158508, 'FPR95TPR': 0.1459999978542328}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.8990402221679688, 'AUPR-IN': 0.9661892652511597, 'AUPR-OUT': 0

### DMR

In [72]:
DMR_path = "/home/dasol/DMR/DMR/CIFAR10_OOD_training_images_using_DMR_single"
#DMR_path = "/home/dasol/DMR/DMR/CIFAR10_OOD_training_images_using_DMR_multiple6"
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
DMR_single = train_ood_detector(DMR_path, model, batch_size=128, epochs=10)

Files already downloaded and verified
CAM-based OOD NUM : 50000
Files already downloaded and verified


[train 010/010]: 100%|##############################################| 390/390 [00:04<00:00, 78.22it/s]


In [73]:
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
detector = Mine(model, DMR_single)
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_AUPR = np.mean([r['AUPR-IN'] for r in results])

print("AUROC:", mean_AUROC)
print("AURR:", mean_AUPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.99465012550354, 'AUPR-IN': 0.9979754686355591, 'AUPR-OUT': 0.986281156539917, 'FPR95TPR': 0.021400000900030136}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.9581211805343628, 'AUPR-IN': 0.9403345584869385, 'AUPR-OUT': 0.9693204164505005, 'FPR95TPR': 0.2093999981880188}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9959545135498047, 'AUPR-IN': 0.9955841898918152, 'AUPR-OUT': 0.9962907433509827, 'FPR95TPR': 0.01850000023841858}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.981687068939209, 'AUPR-IN': 0.9802305698394775, 'AUPR-OUT': 0.9831022024154663, 'FPR95TPR': 0.08129999786615372}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.9329447150230408, 'AUPR-IN': 0.9786102175712585, 'AUPR-OUT': 0.8

In [79]:
KIRBY_path = "/home/dasol/DMR/DMR/CIFAR10_OOD_training_images_using_KIRBY"
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
KIRBY = train_ood_detector(KIRBY_path, model, batch_size=128, epochs=10, seed=42)

Files already downloaded and verified
CAM-based OOD NUM : 50000
Files already downloaded and verified


[train 010/010]: 100%|##############################################| 390/390 [00:04<00:00, 79.98it/s]


In [80]:
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
detector = Mine(model, KIRBY)
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_AUPR = np.mean([r['AUPR-IN'] for r in results])

print("AUROC:", mean_AUROC)
print("AURR:", mean_AUPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.990415632724762, 'AUPR-IN': 0.9963837265968323, 'AUPR-OUT': 0.9748173952102661, 'FPR95TPR': 0.04270000010728836}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.952820897102356, 'AUPR-IN': 0.9385427832603455, 'AUPR-OUT': 0.9624128937721252, 'FPR95TPR': 0.26260000467300415}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9960233569145203, 'AUPR-IN': 0.9957878589630127, 'AUPR-OUT': 0.9963030219078064, 'FPR95TPR': 0.016300000250339508}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9811729788780212, 'AUPR-IN': 0.9803964495658875, 'AUPR-OUT': 0.9820458292961121, 'FPR95TPR': 0.08370000123977661}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.9334304332733154, 'AUPR-IN': 0.9789160490036011, 'AUPR-OUT': 

### KIRBY + DMR

In [91]:
KIRBY_DMR_path = ["/home/dasol/DMR/DMR/CIFAR10_OOD_training_images_using_KIRBY",
                  "/home/dasol/DMR/DMR/CIFAR10_OOD_training_images_using_DMR_single"]
model = WideResNet(num_classes=10, pretrained="cifar10-pt").cuda().eval()
KIRBY_DMR = train_ood_detector(KIRBY_DMR_path, model, batch_size=128, epochs=10, seed=42)

Files already downloaded and verified
CAM-based OOD NUM : 100000
Files already downloaded and verified


[train 010/010]: 100%|##############################################| 390/390 [00:05<00:00, 65.60it/s]


In [92]:
dataset_in_test = CIFAR10(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = Mine(model, KIRBY_DMR)
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_AUPR = np.mean([r['AUPR-IN'] for r in results])

print("AUROC:", mean_AUROC)
print("AURR:", mean_AUPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.9939753413200378, 'AUPR-IN': 0.997723400592804, 'AUPR-OUT': 0.9845340847969055, 'FPR95TPR': 0.024299999698996544}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.9628608226776123, 'AUPR-IN': 0.9482598304748535, 'AUPR-OUT': 0.9719141125679016, 'FPR95TPR': 0.18230000138282776}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9961293339729309, 'AUPR-IN': 0.9958504438400269, 'AUPR-OUT': 0.9964372515678406, 'FPR95TPR': 0.016599999740719795}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9816505908966064, 'AUPR-IN': 0.9805824756622314, 'AUPR-OUT': 0.9828194379806519, 'FPR95TPR': 0.08460000157356262}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.9338406324386597, 'AUPR-IN': 0.9789539575576782, 'AUPR-OUT'